# Debugging autoreload

In [ ]:
import optuna
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import shutil
import pickle
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import optuna
import pathlib

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*exists and is not empty.*")
warnings.filterwarnings("ignore", ".*is smaller than the logging interval Trainer.*")


# Load data

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)' # 'origin' 'imp_source(imm)_method(knn)_params(5)' 'imp_source(imm)_method(miceforest)_params(2)'

selection_method = 'mrmr' # 'f_regression' 'spearman' 'mrmr'
n_feats = 100

imm = 'CX3CL1'

tst_n_splits = 5
tst_n_repeats = 5
tst_random_state = 1337
tst_split_id = 5

val_n_splits = 4
val_n_repeats = 2
val_random_state = 1337
val_fold_id = 5

fn_samples = f"samples_tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats})"
with open(f"D:/YandexDisk/Work/bbd/immunology/003_EpImAge/{fn_samples}.pickle", 'rb') as handle:
    samples = pickle.load(handle)
    
for split_id in range(tst_n_splits * tst_n_repeats):
    for fold_id in range(val_n_splits * val_n_repeats):
        test_samples = samples[split_id]['test']
        train_samples = samples[split_id]['trains'][fold_id]
        validation_samples = samples[split_id]['validations'][fold_id]

        intxns = {
            'train_validation': set.intersection(set(train_samples), set(validation_samples)),
            'validation_test': set.intersection(set(validation_samples), set(test_samples)),
            'train_test': set.intersection(set(train_samples), set(test_samples))
        }
        
        for intxn_name, intxn_samples in intxns.items():
            if len(intxn_samples) > 0:
                print(f"Non-zero {intxn_name} intersection ({len(intxn_samples)}) for {split_id} Split and {fold_id} Fold!")

path_data = f"D:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/{imm}"
pathlib.Path(f"{path_data}/pytorch_tabular").mkdir(parents=True, exist_ok=True)
path_configs = "D:/Work/bbs/notebooks/immunology/003_EpImAge/immuno_regression_configs"
data = pd.read_excel(f"{path_data}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path_data}/feats_con.xlsx", index_col=0).index.values.tolist()

split_dict = samples[tst_split_id]

test = data.loc[split_dict['test'], feats + [f"{imm}_log"]]
train = data.loc[split_dict['trains'][val_fold_id], feats + [f"{imm}_log"]]
validation = data.loc[split_dict['validations'][val_fold_id], feats + [f"{imm}_log"]]

## Load non-model configs

In [ ]:
data_config = read_parse_config(f"{path_configs}/DataConfig.yaml", DataConfig)
data_config['target'] = [f"{imm}_log"]
data_config['continuous_cols'] = feats
trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
trainer_config['checkpoints_path'] = f"{path_data}/pytorch_tabular"
optimizer_config = read_parse_config(f"{path_configs}/OptimizerConfig.yaml", OptimizerConfig)

lr_find_min_lr = 1e-8
lr_find_max_lr = 10
lr_find_num_training = 512
lr_find_mode = "exponential"
lr_find_early_stop_threshold = 8.0

# Optuna Hyperparameter Optimization

## Init default model config, datamodule

In [ ]:
seed = 1337
model_name = 'DANet'

n_trials = 64
opt_seed = 40
n_startup_trials = 32
n_ei_candidates = 16

trainer_config['seed'] = seed
trainer_config['checkpoints'] = 'valid_loss'
trainer_config['load_best'] = True
trainer_config['auto_lr_find'] = False

model_config_default = read_parse_config(f"{path_configs}/models/{model_name}Config.yaml", GANDALFConfig)
tabular_model_default = TabularModel(
    data_config=data_config,
    model_config=model_config_default,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=False,
)
datamodule = tabular_model_default.prepare_dataloader(train=train, validation=validation, seed=seed)

## Init optimization metrics with directions

In [ ]:
opt_parts = ['test', 'validation']
opt_metrics = [('mean_absolute_error', 'minimize'), ('pearson_corrcoef', 'maximize')]
opt_directions = []
for part in opt_parts:
    for metric_pair in opt_metrics:
        opt_directions.append(f"{metric_pair[1]}")

## Model Tuner, No Cross-Validation (Train, Validation, Test)

In [ ]:
%%capture

trials_results = []

study = optuna.create_study(
    study_name=model_name,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=n_startup_trials,
        n_ei_candidates=n_ei_candidates,
        seed=opt_seed,
    ),
    directions=opt_directions
)
study.optimize(
    func=lambda trial: train_hyper_opt(
        trial=trial,
        trials_results=trials_results,
        opt_metrics=opt_metrics,
        opt_parts=opt_parts,
        model_config_default=model_config_default,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        experiment_config=None,
        train=train,
        validation=validation,
        test=test,
        datamodule=datamodule,
        min_lr=lr_find_min_lr,
        max_lr=lr_find_max_lr,
        num_training=lr_find_num_training,
        mode=lr_find_mode,
        early_stop_threshold=lr_find_early_stop_threshold
    ), 
    n_trials=n_trials, 
    show_progress_bar=True
)

In [ ]:
fn_trials = (f"model({model_name})_trials({n_trials}_{opt_seed}_{n_startup_trials}_{n_ei_candidates})_"
             f"tst({tst_split_id})_val({val_fold_id})_"
             f"{optimizer_config['lr_scheduler']}_{data_config['continuous_feature_transform']}")

df_trials = pd.DataFrame(trials_results)
df_trials['split_id'] = tst_split_id
df_trials['fold_id'] = val_fold_id
df_trials["train_more"] = False
df_trials.loc[(df_trials["train_loss"] > df_trials["test_loss"]) | (
            df_trials["train_loss"] > df_trials["validation_loss"]), "train_more"] = True
df_trials["validation_test_mean_loss"] = (df_trials["validation_loss"] + df_trials["test_loss"]) / 2.0
df_trials["train_validation_test_mean_loss"] = (df_trials["train_loss"] + df_trials["validation_loss"] + df_trials["test_loss"]) / 3.0

df_trials.sort_values(by=['test_loss'], ascending=[True], inplace=True)
df_trials.style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "validation_test_mean_loss",
        "train_validation_test_mean_loss",
        "test_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{trainer_config['checkpoints_path']}/{fn_trials}.xlsx")

# Model Sweep Training

## Models Search Spaces

### GANDALF Search Space

In [ ]:
search_space = {
    "model_config__gflu_stages": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "model_config__gflu_dropout": [0.0, 0.1],
    "model_config__gflu_feature_init_sparsity": [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5],
    "model_config.head_config__dropout": [0.0, 0.1],
    "model_config__learning_rate": [0.001],
    "model_config__seed": [1337],
}
grid_size = np.prod([len(p_vals) for _, p_vals in search_space.items()])
print(grid_size)

head_config = LinearHeadConfig(
    layers="",
    activation='ReLU',
    dropout=0.1,
    use_batch_norm=False,
    initialization="kaiming",
).__dict__

model_list = []
for i, params in enumerate(ParameterGrid(search_space)):
    head_config_tmp = copy.deepcopy(head_config)
    head_config_tmp['dropout'] = params['model_config.head_config__dropout']
    model_config = read_parse_config(f"{path_configs}/models/GANDALFConfig.yaml", GANDALFConfig)
    model_config['gflu_stages'] = params['model_config__gflu_stages']
    model_config['gflu_feature_init_sparsity'] = params['model_config__gflu_feature_init_sparsity']
    model_config['gflu_dropout'] = params['model_config__gflu_dropout']
    model_config['learning_rate'] = params['model_config__learning_rate']
    model_config['seed'] = params['model_config__seed']
    model_config['head_config'] = head_config_tmp
    model_list.append(GANDALFConfig(**model_config))

### DANet Search Space

In [ ]:
search_space = {
    "model_config__n_layers": [4, 6, 8, 10, 12, 14, 16],
    "model_config__abstlay_dim_1": [8, 16, 32],
    "model_config__k": [3, 4, 5, 6],
    "model_config__dropout_rate": [0.0, 0.1],
    "model_config.head_config__dropout": [0.0, 0.1],
    "model_config__learning_rate": [0.001],
    "model_config__seed": [1337],
}
grid_size = np.prod([len(p_vals) for _, p_vals in search_space.items()])
print(grid_size)

head_config = LinearHeadConfig(
    layers="",
    activation='ReLU',
    dropout=0.1,
    use_batch_norm=False,
    initialization="kaiming"
).__dict__

model_list = []
for i, params in enumerate(ParameterGrid(search_space)):
    head_config_tmp = copy.deepcopy(head_config)
    head_config_tmp['dropout'] = params['model_config.head_config__dropout']
    model_config = read_parse_config(f"{path_configs}/models/DANetConfig.yaml", DANetConfig)
    model_config['n_layers'] = params['model_config__n_layers']
    model_config['abstlay_dim_1'] = params['model_config__abstlay_dim_1']
    model_config['k'] = params['model_config__k']
    model_config['dropout_rate'] = params['model_config__dropout_rate']
    model_config['learning_rate'] = params['model_config__learning_rate']
    model_config['seed'] = params['model_config__seed']
    model_config['head_config'] = head_config_tmp
    model_list.append(DANetConfig(**model_config))

## Perform model sweep

In [ ]:
%%capture

common_params = {
    "task": "regression",
}

seeds = [1337]  # [1337, 55763, 40279, 87571, 234461]

dfs_result = []
for seed in seeds:

    trainer_config['seed'] = seed
    trainer_config['checkpoints'] = 'valid_loss'
    trainer_config['load_best'] = True
    trainer_config['auto_lr_find'] = True

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sweep_df, best_model = model_sweep_custom(
            task="regression",
            train=train,
            validation=validation,
            test=test,
            data_config=data_config,
            optimizer_config=optimizer_config,
            trainer_config=trainer_config,
            model_list=model_list,
            common_model_args=common_params,
            metrics=["mean_absolute_error", "pearson_corrcoef"],
            metrics_params=[{}, {}],
            metrics_prob_input=[False, False],
            rank_metric=("mean_absolute_error", "lower_is_better"),
            return_best_model=True,
            seed=seed,
            progress_bar=False,
            verbose=False,
            suppress_lightning_logger=True,
            min_lr=lr_find_min_lr,
            max_lr=lr_find_max_lr,
            num_training=lr_find_num_training,
            mode=lr_find_mode,
            early_stop_threshold=lr_find_early_stop_threshold,
        )
    sweep_df['seed'] = seed
    sweep_df['split_id'] = tst_split_id
    sweep_df['fold_id'] = val_fold_id
    sweep_df["train_more"] = False
    sweep_df.loc[(sweep_df["train_loss"] > sweep_df["test_loss"]) | (
                sweep_df["train_loss"] > sweep_df["validation_loss"]), "train_more"] = True
    sweep_df["validation_test_mean_loss"] = (sweep_df["validation_loss"] + sweep_df["test_loss"]) / 2.0
    sweep_df["train_validation_test_mean_loss"] = (sweep_df["train_loss"] + sweep_df["validation_loss"] + sweep_df["test_loss"]) / 3.0

    dfs_result.append(sweep_df)

    fn_suffix = (f"models({len(model_list)})_"
                 f"tst({tst_split_id})_val({val_fold_id})_"
                 f"{best_model.config['lr_scheduler']}_{best_model.config['continuous_feature_transform']}")
    try:
        df_result = pd.concat(dfs_result, ignore_index=True)
        df_result.sort_values(by=['test_loss'], ascending=[True], inplace=True)
        df_result.style.background_gradient(
            subset=[
                "train_loss",
                "validation_loss",
                "validation_test_mean_loss",
                "train_validation_test_mean_loss",
                "test_loss",
                "time_taken",
                "time_taken_per_epoch"
            ], cmap="RdYlGn_r"
        ).to_excel(f"{trainer_config['checkpoints_path']}/{fn_suffix}.xlsx")
    except PermissionError:
        pass

# Save best models (legacy)

In [ ]:
%%capture

fn_trials = (f"model({model_name})_trials({n_trials}_{opt_seed}_{n_startup_trials}_{n_ei_candidates})_"
             f"tst({tst_split_id})_val({val_fold_id})_"
             f"{optimizer_config['lr_scheduler']}")

df_sweeps = pd.read_excel(f"{trainer_config['checkpoints_path']}/{fn_trials}.xlsx", index_col=0)
path_models = f"{trainer_config['checkpoints_path']}/candidates/{fn_trials}"
pathlib.Path(path_models).mkdir(parents=True, exist_ok=True)
df_sweeps.style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "test_loss",
        "validation_test_mean_loss",
        "train_validation_test_mean_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{path_models}/sweep.xlsx")

models_ids = [
43,
17,
42,
48,
47,
]
models_ids = list(set(models_ids))

df_sweeps.loc[models_ids, :].style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "test_loss",
        "validation_test_mean_loss",
        "train_validation_test_mean_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{path_models}/selected.xlsx")

explain_method = "GradientShap"
explain_baselines = "b|1000"
explain_n_feats_to_plot = 25

for model_id in models_ids:

    tabular_model = TabularModel(
        data_config=ast.literal_eval(df_sweeps.at[model_id, 'data_params']),
        model_config=ast.literal_eval(df_sweeps.at[model_id, 'model_params']),
        optimizer_config=ast.literal_eval(df_sweeps.at[model_id, 'optimizer_params']),
        trainer_config=trainer_config,
        verbose=True,
        suppress_lightning_logger=True
    )
    datamodule = tabular_model.prepare_dataloader(
        train=train,
        validation=validation,
        seed=trainer_config['seed'],
    )
    model = tabular_model.prepare_model(
        datamodule
    )
    tabular_model._prepare_for_training(
        model,
        datamodule
    )
    tabular_model.load_weights(df_sweeps.at[model_id, 'checkpoint'])
    tabular_model.evaluate(test, verbose=False)
    tabular_model.save_model(f"{path_models}/{model_id}")
    
    loaded_model = TabularModel.load_model(f"{path_models}/{model_id}")
    
    df = data.loc[:, data_config['target']]
    df.loc[train.index, 'Group'] = 'Train'
    df.loc[validation.index, 'Group'] = 'Validation'
    df.loc[test.index, 'Group'] = 'Test'
    df['Prediction'] = loaded_model.predict(data)
    df['Error'] = df['Prediction'] - df[data_config['target'][0]]
    df.to_excel(f"{path_models}/{model_id}/df.xlsx")
    
    colors_groups = {
        'Train': 'chartreuse',
        'Validation': 'dodgerblue',
        'Test': 'crimson',
    }
    
    df_metrics = pd.DataFrame(
        index=list(colors_groups.keys()),
        columns=['mean_absolute_error', 'pearson_corrcoef', 'bias']
    )
    for group in colors_groups.keys():
        pred = torch.from_numpy(df.loc[df['Group'] == group, 'Prediction'].values)
        real = torch.from_numpy(df.loc[df['Group'] == group, data_config['target'][0]].values)
        df_metrics.at[group, 'mean_absolute_error'] = mean_absolute_error(pred, real).numpy()
        df_metrics.at[group, 'pearson_corrcoef'] = pearson_corrcoef(pred, real).numpy()
        df_metrics.at[group, 'bias'] = np.mean(df.loc[df['Group'] == group, 'Error'].values)
    df_metrics.to_excel(f"{path_models}/{model_id}/metrics.xlsx", index_label="Metrics")
    
    sns.set_theme(style='whitegrid')
    xy_min = df[[data_config['target'][0], 'Prediction']].min().min()
    xy_max = df[[data_config['target'][0], 'Prediction']].max().max()
    xy_ptp = xy_max - xy_min
    fig, ax = plt.subplots(figsize=(4.5, 4))
    scatter = sns.scatterplot(
        data=df,
        x=data_config['target'][0],
        y="Prediction",
        hue="Group",
        palette=colors_groups,
        linewidth=0.2,
        alpha=0.75,
        edgecolor="k",
        s=20,
        hue_order=list(colors_groups.keys()),
        ax=ax
    )
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=ax
    )
    ax.set_title(f"{df_sweeps.at[model_id, 'model']} ({df_sweeps.at[model_id, '# Params']} params, {df_sweeps.at[model_id, 'epochs']} epochs)")
    ax.set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    ax.set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    plt.gca().set_aspect('equal', adjustable='box')
    fig.savefig(f"{path_models}/{model_id}/scatter.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_models}/{model_id}/scatter.pdf", bbox_inches='tight')
    plt.close(fig)
    
    df_fig = df.loc[:, ['Error', 'Group']]
    groups_rename = {
        group: f"{group}" + "\n" +
               fr"MAE: {df_metrics.at[group, 'mean_absolute_error']:0.2f}" + "\n"
               fr"Pearson $\rho$: {df_metrics.at[group, 'pearson_corrcoef']:0.2f}" + "\n" +
               fr"$\langle$Error$\rangle$: {df_metrics.at[group, 'bias']:0.2f}" 
        for group in colors_groups
    }
    colors_groups_violin = {groups_rename[group]: colors_groups[group] for group in colors_groups}
    df_fig['Group'].replace(groups_rename, inplace=True)
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(7, 4))
    violin = sns.violinplot(
        data=df_fig,
        x='Group',
        y='Error',
        palette=colors_groups_violin,
        scale='width',
        order=list(colors_groups_violin.keys()),
        saturation=0.75,
        legend=False,
        ax=ax
    )
    ax.set_xlabel('')
    fig.savefig(f"{path_models}/{model_id}/violin.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_models}/{model_id}/violin.pdf", bbox_inches='tight')
    plt.close(fig)
    
    try:
        explanation = loaded_model.explain(data, method=explain_method, baselines=explain_baselines)
        explanation.index = data.index
        explanation.to_excel(f"{path_models}/{model_id}/explanation.xlsx")
        
        sns.set_theme(style='whitegrid')
        fig = shap.summary_plot(
            shap_values=explanation.loc[:, feats].values,
            features=data.loc[:, feats].values,
            feature_names=feats,
            max_display=explain_n_feats_to_plot,
            plot_type="violin",
            show=False,
        )
        plt.savefig(f"{path_models}/{model_id}/explain_beeswarm.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_models}/{model_id}/explain_beeswarm.pdf", bbox_inches='tight')
        plt.close(fig)
        
        sns.set_theme(style='whitegrid')
        fig = shap.summary_plot(
            shap_values=explanation.loc[:, feats].values,
            features=data.loc[:, feats].values,
            feature_names=feats,
            max_display=explain_n_feats_to_plot,
            plot_type="bar",
            show=False,
        )
        plt.savefig(f"{path_models}/{model_id}/explain_bar.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_models}/{model_id}/explain_bar.pdf", bbox_inches='tight')
        plt.close(fig)
    
    except NotImplementedError:
        pass

# Save best models

In [ ]:
%%capture

models_names = [
    'FTTransformerModel',
    'DANetModel',
    'GANDALFModel',
    'CategoryEmbeddingModel',
    'TabNetModel'
]

target = f"{imm}_log"

path_ckpts = f'{path_data}/pytorch_tabular'

fn_trials = (f"models_"
             f"tst({tst_split_id})_"
             f"val({val_fold_id})")
df_trials = pd.read_excel(f"{path_ckpts}/{fn_trials}.xlsx", index_col=0)
pathlib.Path(f"{path_ckpts}/candidates").mkdir(parents=True, exist_ok=True)
df_trials.style.background_gradient(
    subset=[
        "train_loss",
        "validation_loss",
        "validation_test_mean_loss",
        "train_validation_test_mean_loss",
        "test_loss",
        "time_taken",
        "time_taken_per_epoch"
    ], cmap="RdYlGn_r"
).to_excel(f"{path_ckpts}/candidates/{fn_trials}.xlsx")

for model_name in models_names:
    print(model_name)
    df_model_all = df_trials[df_trials['model'] == model_name]
    path_model = f"{path_ckpts}/candidates/{model_name}"
    pathlib.Path(path_model).mkdir(parents=True, exist_ok=True)
    df_model_all.style.background_gradient(
        subset=[
            "train_loss",
            "validation_loss",
            "test_loss",
            "validation_test_mean_loss",
            "train_validation_test_mean_loss",
            "time_taken",
            "time_taken_per_epoch"
        ], cmap="RdYlGn_r"
    ).to_excel(f"{path_model}/all.xlsx")
    
    df_model_selected = df_model_all[df_model_all['Selected'] == 1]
    df_model_selected.style.background_gradient(
        subset=[
            "train_loss",
            "validation_loss",
            "test_loss",
            "validation_test_mean_loss",
            "train_validation_test_mean_loss",
            "time_taken",
            "time_taken_per_epoch"
        ], cmap="RdYlGn_r"
    ).to_excel(f"{path_model}/selected.xlsx")
    
    explain_method = "GradientShap"
    explain_baselines = "b|1000"
    explain_n_feats_to_plot = 25
    
    for model_id, row in df_model_selected.iterrows():
        
        model_dir = (str(pathlib.Path(row['checkpoint']).parent).replace('\\', '/') 
                     + '/' + pathlib.Path(row['checkpoint']).stem)
        
        model = TabularModel.load_model(model_dir)
        pathlib.Path(f"{path_model}/{model_id}").mkdir(parents=True, exist_ok=True)
        shutil.copytree(model_dir, f"{path_model}/{model_id}", dirs_exist_ok=True)
        
        df = data.loc[:, [target]]
        df.loc[train.index, 'Group'] = 'Train'
        df.loc[validation.index, 'Group'] = 'Validation'
        df.loc[test.index, 'Group'] = 'Test'
        df['Prediction'] = model.predict(data)
        df['Error'] = df['Prediction'] - df[target]
        df.to_excel(f"{path_model}/{model_id}/df.xlsx")
        
        colors_groups = {
            'Train': 'chartreuse',
            'Validation': 'dodgerblue',
            'Test': 'crimson',
        }
        
        df_metrics = pd.DataFrame(
            index=list(colors_groups.keys()),
            columns=['mean_absolute_error', 'pearson_corrcoef', 'bias']
        )
        for group in colors_groups.keys():
            pred = torch.from_numpy(df.loc[df['Group'] == group, 'Prediction'].values)
            real = torch.from_numpy(df.loc[df['Group'] == group, target].values)
            df_metrics.at[group, 'mean_absolute_error'] = mean_absolute_error(pred, real).numpy()
            df_metrics.at[group, 'pearson_corrcoef'] = pearson_corrcoef(pred, real).numpy()
            df_metrics.at[group, 'bias'] = np.mean(df.loc[df['Group'] == group, 'Error'].values)
        df_metrics.to_excel(f"{path_model}/{model_id}/metrics.xlsx", index_label="Metrics")
        
        sns.set_theme(style='whitegrid')
        xy_min = df[[target, 'Prediction']].min().min()
        xy_max = df[[target, 'Prediction']].max().max()
        xy_ptp = xy_max - xy_min
        fig, ax = plt.subplots(figsize=(4.5, 4))
        scatter = sns.scatterplot(
            data=df,
            x=target,
            y="Prediction",
            hue="Group",
            palette=colors_groups,
            linewidth=0.2,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(colors_groups.keys()),
            ax=ax
        )
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=ax
        )
        ax.set_title(f"{row['model']} ({row['# Params']} params, {row['epochs']} epochs)")
        ax.set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        ax.set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        plt.gca().set_aspect('equal', adjustable='box')
        fig.savefig(f"{path_model}/{model_id}/scatter.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path_model}/{model_id}/scatter.pdf", bbox_inches='tight')
        plt.close(fig)
        
        df_fig = df.loc[:, ['Error', 'Group']]
        groups_rename = {
            group: f"{group}" + "\n" +
                   fr"MAE: {df_metrics.at[group, 'mean_absolute_error']:0.2f}" + "\n"
                   fr"Pearson $\rho$: {df_metrics.at[group, 'pearson_corrcoef']:0.2f}" + "\n" +
                   fr"$\langle$Error$\rangle$: {df_metrics.at[group, 'bias']:0.2f}" 
            for group in colors_groups
        }
        colors_groups_violin = {groups_rename[group]: colors_groups[group] for group in colors_groups}
        df_fig['Group'].replace(groups_rename, inplace=True)
        sns.set_theme(style='whitegrid')
        fig, ax = plt.subplots(figsize=(7, 4))
        violin = sns.violinplot(
            data=df_fig,
            x='Group',
            y='Error',
            palette=colors_groups_violin,
            scale='width',
            order=list(colors_groups_violin.keys()),
            saturation=0.75,
            legend=False,
            ax=ax
        )
        ax.set_xlabel('')
        fig.savefig(f"{path_model}/{model_id}/violin.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path_model}/{model_id}/violin.pdf", bbox_inches='tight')
        plt.close(fig)
        
        try:
            explanation = model.explain(data, method=explain_method, baselines=explain_baselines)
            explanation.index = data.index
            explanation.to_excel(f"{path_model}/{model_id}/explanation.xlsx")
            
            sns.set_theme(style='whitegrid')
            fig = shap.summary_plot(
                shap_values=explanation.loc[:, feats].values,
                features=data.loc[:, feats].values,
                feature_names=feats,
                max_display=explain_n_feats_to_plot,
                plot_type="violin",
                show=False,
            )
            plt.savefig(f"{path_model}/{model_id}/explain_beeswarm.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_model}/{model_id}/explain_beeswarm.pdf", bbox_inches='tight')
            plt.close(fig)
            
            sns.set_theme(style='whitegrid')
            fig = shap.summary_plot(
                shap_values=explanation.loc[:, feats].values,
                features=data.loc[:, feats].values,
                feature_names=feats,
                max_display=explain_n_feats_to_plot,
                plot_type="bar",
                show=False,
            )
            plt.savefig(f"{path_model}/{model_id}/explain_bar.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_model}/{model_id}/explain_bar.pdf", bbox_inches='tight')
            plt.close(fig)
        
        except NotImplementedError:
            pass